### Implementation of GRU on the time-series dataset

In [23]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [24]:
# Load test features and labels
X_test = np.load('rolling_window_sequences.npy')  # Replace with actual file path
metadata_test = pd.read_csv("sequence_metadata_with_RUL.csv")  # Replace with actual file path
y_test = metadata_test["RUL"].values
print("Test feature shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Test feature shape: (17631, 30, 66)
Test target shape: (17631,)


In [25]:
# Split the data into train and validation sets (assuming no separate train set given)
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

In [26]:
# Define GRU model creation function
def create_gru_model(input_shape, units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [27]:
# Create the model
model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]), units=64, learning_rate=0.001, dropout_rate=0.2)

# Callbacks for early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

c:\Users\win10\python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 7619.3813 - mae: 76.0429 - val_loss: 5854.6714 - val_mae: 64.8472
Epoch 2/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 5497.6406 - mae: 62.4432 - val_loss: 4534.1670 - val_mae: 55.4343
Epoch 3/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 4244.8384 - mae: 53.2952 - val_loss: 3528.2424 - val_mae: 47.8430
Epoch 4/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3336.0378 - mae: 46.5107 - val_loss: 2747.3362 - val_mae: 41.5641
Epoch 5/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2621.5093 - mae: 40.6729 - val_loss: 2144.2107 - val_mae: 36.4552
Epoch 6/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2047.4005 - mae: 35.6209 - val_loss: 1673.6516 - val_mae: 32.0420
Epoch 7/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1573.0201 - mae: 31.2573 - val_loss: 1285.1062 - val_mae: 28.0503
Epoch 8/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1234.8544 - mae: 27.8473 - val_loss: 993.8656 - va

In [29]:
# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation MSE: {val_loss:.4f}, MAE: {val_mae:.4f}')

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Calculate test MSE
test_mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {test_mse:.4f}')

Validation MSE: 118.2111, MAE: 8.0871
551/551 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Test MSE: 102.2187


Task: Build and train a GRU (Gated Recurrent Unit) neural network model for sequence regression on a given dataset. The implementation must include the following features:

Use of callbacks such as EarlyStopping to prevent overfitting and ModelCheckpoint to save the best model.

Hyperparameter tuning for model parameters like number of GRU units, learning rate, and dropout rate.

Cross-validation (e.g., K-Fold) to robustly evaluate model performance across different data splits.

Reporting of evaluation metrics such as mean squared error (MSE) and mean absolute error (MAE).